#### Notebook `Feature Selection`

#### Group:
- `Miguel Matos - 20221925`
- `André Nicolau - 20221861`
- `André Ferreira - 20250398`

---

#### <font> Table of Contents </font> <a class="anchor" id='toc'></a> 
1. [Imports](#Imports)
2. [Feature Selection](#feature-selection)


# Imports
[Back to TOC](#toc)

In [1]:
from functions import *

In [7]:
X_train = pd.read_csv("/Users/miguelmatos/Documents/GitHub/Machine_Learning_project/ML_Project_Group42/data/X_train.csv", index_col= "carID")
y_train = pd.read_csv("/Users/miguelmatos/Documents/GitHub/Machine_Learning_project/ML_Project_Group42/data/y_train.csv", index_col= "carID")

X_val = pd.read_csv("/Users/miguelmatos/Documents/GitHub/Machine_Learning_project/ML_Project_Group42/data/X_val.csv", index_col= "carID")
y_val = pd.read_csv("/Users/miguelmatos/Documents/GitHub/Machine_Learning_project/ML_Project_Group42/data/y_val.csv", index_col= "carID")

pd.set_option("display.max_columns", None)

# Feature Selection
